In [1]:
from kafka import KafkaProducer,KafkaClient
from kafka.errors import KafkaError
import time
import json
import websocket
import logging
import re
#LOCAL IMPORTS
import config as myCred
from confluent_kafka.admin import AdminClient

In [3]:
#unit test to check if zookeeper an kafka are running properly
conf = {'bootstrap.servers':'localhost:9092'}
admin_client = AdminClient(conf)
topics = admin_client.list_topics().topics

if not topics: 
    raise RuntimeError()
else:
    print(topics)

{'stockData': TopicMetadata(stockData, 1 partitions), 'JivWn': TopicMetadata(JivWn, 1 partitions), '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}


In [32]:
#functions for API connnection

startTime =time.time()

def onOpen(ws):
    print("connection Opened")
    auth_data={
        "action": "authenticate",
        "data": {"key_id": myCred.API_KEY, "secret_key": myCred.SECRET_KEY}
    }
    
    ws.send(json.dumps(auth_data))
    
    listenMsg = {
        "action": "listen", 
        "data": {"streams": ["T.TSLA","T.AAPL","T.MSFT"]}
    }

    ws.send(json.dumps(listenMsg))
        
def onMsg(ws,message):
    if re.match('^.*authorized.*$',message):
        return
    else:
        if(time.time()-startTime)<30:
            print(message)
            try:
                producer.send(topicName, message.encode('utf-8'))
                return True
            except Exception as ex:
                print('Exception in publishing message')
                print(str(ex))
        else:
            ws.close()
    
def onClose(ws):
    print("connection Closed")

In [33]:
socket = "wss://data.alpaca.markets/stream"
ws = websocket.WebSocketApp(socket,on_open=onOpen, on_close=onClose, on_message=onMsg)
try:
    producer=KafkaProducer(bootstrap_servers=" localhost:9092")
    topicName="stockData"
except KafkaError:
    print(logging.exception("producer run time error"))
    

In [34]:
ws.run_forever()

connection Opened
{"stream":"listening","data":{"streams":["T.TSLA","T.AAPL","T.MSFT"]}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"905","x":17,"p":211.26,"s":2,"t":1594217502114000000,"c":[37],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"778","x":2,"p":379.26,"s":82,"t":1594217502168000000,"c":[14,37,41],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"779","x":2,"p":379.26,"s":18,"t":1594217502168000000,"c":[14,37,41],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"457","x":15,"p":1403.74,"s":20,"t":1594217502156000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"458","x":15,"p":1403.65,"s":20,"t":1594217502156000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"459","x":15,"p":1403.68,"s":38,"t":1594217502157000000,"c":[37],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"539","x":15,"p":211.31,"s":100,"t":1594217505822000000,"c":null,"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","